# Evaluation

## P@K

$P@k=\dfrac{TP@k}{TP@k+FP@k}$

In [1]:
from llms4subjects.instance import EmbeddingQuery as EmbeddingQuery
from llms4subjects.subject import subject_db_all as subject_db
from llms4subjects.util import rm_leading_blanks

def make_examples(title:str, abstract:str, dataset_type:str="all", topk = 5) -> str:
    text = f"""title:{title}\nabstract:{abstract}"""
    eq = EmbeddingQuery(f"./db/instance/{dataset_type}")
    instances = eq.get_instances(text, topk)
    examples = []
    for i, inst in enumerate(instances, start=1):
        gnd_names = [subject_db.get_name_by_code(c) for c in inst.gnd_codes]
        words = inst.abstract.split(" ")
        if len(words) > 500:
            words = words[0:500]
            words.append("...")
        words = " ".join(words)
        names = ", ".join([f'"{name}"' for name in gnd_names])
        examples.append(rm_leading_blanks(f"""## Example {i}
        - Title: ```{inst.title}```
        - Abstract: ```{words}``` 
        - Subject: [{names}]
        """))
        
    return "\n".join(examples)

In [2]:
from llms4subjects.simple_predict import get_dev_dataset

samples = get_dev_dataset("core")
sample = samples[0]
title, abstract = sample["title"], sample["abstract"]
subjects = sample["true_names"]

  0%|          | 0/6980 [00:00<?, ?it/s]

100%|██████████| 6980/6980 [00:00<00:00, 15617.42it/s]


In [3]:
examples = make_examples(title, abstract)
print(examples)

## Example 1
- Title: ```Macht und Ohnmacht sozialpädagogischer Hilfe : biografische Perspektiven auf pädagogisch begleitete Übergänge```
- Abstract: ```For disadvantaged young people, (social) pedagogical help in the transition from school to (vocational) training is of great importance in shaping these transitions. A longitudinal study will focus on the young people participating in transition assistance and question how (social) educational assistance can become biographically relevant for them. The transitions constitute a process between biographical processes of appropriation and education and institutional regulation. Für benachteiligte Jugendliche besitzt (sozial-)pädagogische Hilfe im Übergang von der Schule in die Ausbildung eine hohe Bedeutung für die Gestaltung dieser Übergänge. Mittels einer Längsschnittstudie wird der Blick auf die an Übergangshilfen teilnehmenden Jugendlichen gerichtet und gefragt, wie (sozial-)pädagogische Hilfe für Jugendliche biografisch relevant werd

In [4]:

def make_prompt(title:str, abstract:str) -> str:
    examples = make_examples(title, abstract)
    words = abstract.split(" ")
    if len(words) > 500:
        words = words[0:500]
        words.append("...")
    words = " ".join(words)
    
    return rm_leading_blanks(f"""You are a librarian responsible for assigning a set of subject tags to technical documents based on their titles and abstracts. Here are some examples:

    {examples}

    Now, please complete the subject list based on the following title and abstract:
    - Title: {title}
    - Abstract: {words}
    - Subjects: 
    """)

In [5]:
from llms4subjects.llm import LLM
import json
# from openai import OpenAI

# base_url = "http://intra-umei-endeep-llm-beta.midu.cc/v1"
# api_key = "k"
# model_name = "llama-3-8b-instruct-awq"
# client = OpenAI(base_url=base_url,api_key=api_key)

base_url = "http://10.96.1.43:3087/v1"
api_key = None
model_name = "/data/app/yangyahe/base_model/LLM-Research-Meta-Llama-3.1-8B-Instruct-AWQ-INT4"


client = LLM(base_url, model=model_name)


def gec_messages(text):
    # res_format = "corrected_text:"
    messages = [
        {"role": "system", "content": "You are an English grammar and error checker, please correct the following text. Please output only the corrected result and do not output other unnecessary information"},
        {"role": "user", "content": "Input: hhaha, how are you?"},
        {"role": "assistant", "content": "Output: hhaha, hoW are you?"}, 
        {"role": "user", "content": "Input: {}".format(text)},
    ]
    return messages
 
def chat(text):
    messages = gec_messages(text)
    # completion = client.chat.completions.create(
    #         model=model_name,
    #         messages=messages,
    #         extra_body=sampling_kwargs,
    #         extra_headers={"podName":"Local", "podNameSpace":"beta-mlc"}
    #         )
    # print(completion)
    #res = completion.choices[0].message.content
    res = client.chat_messages(messages)
    return res

text = make_prompt(title, abstract)
print(text)
print("=============")
data = chat(text)
data = json.loads(data)
data["choices"][0]

You are a librarian responsible for assigning a set of subject tags to technical documents based on their titles and abstracts. Here are some examples:

## Example 1
- Title: ```Macht und Ohnmacht sozialpädagogischer Hilfe : biografische Perspektiven auf pädagogisch begleitete Übergänge```
- Abstract: ```For disadvantaged young people, (social) pedagogical help in the transition from school to (vocational) training is of great importance in shaping these transitions. A longitudinal study will focus on the young people participating in transition assistance and question how (social) educational assistance can become biographically relevant for them. The transitions constitute a process between biographical processes of appropriation and education and institutional regulation. Für benachteiligte Jugendliche besitzt (sozial-)pädagogische Hilfe im Übergang von der Schule in die Ausbildung eine hohe Bedeutung für die Gestaltung dieser Übergänge. Mittels einer Längsschnittstudie wird der Bli

{'index': 0,
 'message': {'role': 'assistant',
  'content': '["Hilfen für junge Menschen", "Selbstbestimmung", "Eigensinn", "Gesellschaftliche Anpassung", "Stationäre Hilfeeinrichtungen", "Alltagspraktiken", "Übergangspraktiken", "Jugendhilfe", "Sozialpolitik", "Rechtspolitik"]',
  'tool_calls': None},
 'logprobs': None,
 'finish_reason': 'stop'}

In [ ]:
from llms4subjects.subject import subject_eq

items = data["choices"][0]["message"]["content"]
items = json.loads(items)
print("title:", title)
print("abstract:", abstract)
print("truth:")
for item in subjects:
    code = subject_db.get_code_by_name(item)
    print("\t",item, ", code:", code)
        
print("predicated:")
for item in items:
    code = subject_eq.get_code_by_name(item)
    name = subject_db.get_name_by_code(code)
    print("\t",name, ", code:", code)

title: Alltags- und Übergangspraktiken in Hilfen für junge Menschen : Zwischen Selbstbestimmung, Eigensinn und gesellschaftlichen Anpassungsforderungen
abstract: Help for young people must be designed in such a way that they have a good chance at the end of the day of being able to live a self-determined life. What does everyday life in stationary aid institutions look like in order to achieve this goal? How can transitions be meaningfully designed? What social and legal policy decisions are necessary? In this volume, everyday and transitional practices in different help systems for young people are examined from different perspectives. Hilfen für junge Menschen müssen so gestaltet sein, dass diese am Ende eine gute Chance haben, selbstbestimmt leben zu können. Wie muss Alltag in stationären Hilfeeinrichtungen aussehen, damit dieses Ziel erreicht wird? Wie können Übergänge sinnvoll gestaltet werden? Welche sozialund rechtspolitischen Weichenstellungen sind notwendig? In diesem Band wer

In [1]:
from llms4subjects.predict.predict_llm import PredictByExamples
from llms4subjects.eval import get_dev_dataset

samples = get_dev_dataset("core")
sample = samples[0]
title, abstract = sample["title"], sample["abstract"]
subjects = sample["true_names"]
predictor = PredictByExamples()
print(title, abstract)


100%|██████████| 6980/6980 [00:00<00:00, 15048.90it/s]

Alltags- und Übergangspraktiken in Hilfen für junge Menschen : Zwischen Selbstbestimmung, Eigensinn und gesellschaftlichen Anpassungsforderungen Help for young people must be designed in such a way that they have a good chance at the end of the day of being able to live a self-determined life. What does everyday life in stationary aid institutions look like in order to achieve this goal? How can transitions be meaningfully designed? What social and legal policy decisions are necessary? In this volume, everyday and transitional practices in different help systems for young people are examined from different perspectives. Hilfen für junge Menschen müssen so gestaltet sein, dass diese am Ende eine gute Chance haben, selbstbestimmt leben zu können. Wie muss Alltag in stationären Hilfeeinrichtungen aussehen, damit dieses Ziel erreicht wird? Wie können Übergänge sinnvoll gestaltet werden? Welche sozialund rechtspolitischen Weichenstellungen sind notwendig? In diesem Band werden Alltags- und 

In [2]:
codes, names = predictor.predict(title, abstract)
print(codes)
print("PREDICT:", names)
print("TRUE:", subjects)

['gnd:4028892-4', 'gnd:4135889-2', 'gnd:4486686-0', 'gnd:4181920-2', 'gnd:4681060-2', 'gnd:4297436-7', 'gnd:4199925-3']
PREDICT: ['Jugendhilfe', 'Selbstbestimmung', 'Eigensinn', 'Soziale Anpassung', 'Übergang', 'Stationäre Behindertenhilfe', 'Jugendsozialarbeit']
TRUE: ['Jugendstrafvollzug', 'Jugendpsychiatrie', 'Erziehungshilfe', 'Jugendhilfe', 'Auslandsaufenthalt', 'Übergang', 'Individualpädagogik']


In [4]:
from llms4subjects.predict.predict_simple import PredictByInstance

predictor2 = PredictByInstance()
codes, names = predictor2.predict(title, abstract)
print(codes)
print(names)

['gnd:4681060-2', 'gnd:4140124-4', 'gnd:4128101-9', 'gnd:4028893-6', 'gnd:4070704-0', 'gnd:4005878-5', 'gnd:4053474-1', 'gnd:4028944-8', 'gnd:4034869-6', 'gnd:4022349-8', 'gnd:4224244-7', 'gnd:4162867-6', 'gnd:4152363-5', 'gnd:4077587-2', 'gnd:4030581-8', 'gnd:4031081-4', 'gnd:4130244-8', 'gnd:4030638-0']
['Übergang', 'Berufsorientierung', 'Empirische Pädagogik', 'Jugendkriminalität', 'Einschulung', 'Berufsberatung', 'Schule', 'Jugendstrafvollzug', 'Lebenslauf', 'Grundschule', 'Dienstleistungsangebot', 'Jugendstrafe', 'Entlassung', 'Sozialer Wandel', 'Kindergarten', 'Kleinkind', 'Kindergartenerziehung', 'Kindertagesstätte']
